In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

05:18:14 INFO:NumExpr defaulting to 8 threads.


In [2]:
week = 0

short_df = run_data_pull(week=week)
# short_df, all_df = run_data_pull(week=week, return_all_games=True)

print(short_df.shape)

05:18:14 DEBUG:Starting new HTTPS connection (1): api.the-odds-api.com:443
05:18:15 DEBUG:https://api.the-odds-api.com:443 "GET /v4/sports/americanfootball_ncaaf/odds?markets=spreads%2Ctotals&regions=us&oddsFormat=american&dateFormat=iso&api_key=ce31cd809f1458374e07be4035c3d210 HTTP/1.1" 200 16706
05:18:15 INFO:Pulled 5 total games for week 0 in 2023
05:18:15 INFO:Picks sheet data has shape (5, 10)
05:18:15 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2023/week0.csv'


(5, 10)


In [3]:
short_df.head()

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
0,0,08/26 06:30 PM (Sat),N,Notre Dame Fighting Irish,Navy Midshipmen,Notre Dame Fighting Irish,Navy Midshipmen,-20.5,49.0,34 - 14
1,0,08/26 09:30 PM (Sat),N,Jacksonville State Gamecocks,UTEP Miners,UTEP Miners,Jacksonville State Gamecocks,-1.5,54.0,27 - 26
2,0,08/26 11:00 PM (Sat),N,New Mexico State Aggies,UMass Minutemen,New Mexico State Aggies,UMass Minutemen,-7.5,45.0,26 - 18
3,0,08/26 11:00 PM (Sat),N,San Diego State Aztecs,Ohio Bobcats,San Diego State Aztecs,Ohio Bobcats,-2.5,48.0,25 - 22
4,0,08/26 11:30 PM (Sat),N,Vanderbilt Commodores,Hawaii Rainbow Warriors,Vanderbilt Commodores,Hawaii Rainbow Warriors,-17.5,56.0,36 - 19


In [4]:
check_team = 'Notre Dame'

short_df[(short_df['Favorite'].str.contains(check_team)) | (short_df['Underdog'].str.contains(check_team))]

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score
0,0,08/26 06:30 PM (Sat),N,Notre Dame Fighting Irish,Navy Midshipmen,Notre Dame Fighting Irish,Navy Midshipmen,-20.5,49.0,34 - 14


In [5]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score


In [6]:
team_list = [
]

all_team_list = short_df['Favorite'].tolist() + short_df['Underdog'].tolist()
mandatory_list = []
for team in team_list:
    for t in all_team_list:
        if t in team:
            mandatory_list.append(t)

mandatory_picks = short_df[(short_df['Favorite'].isin(mandatory_list)) |
                           (short_df['Underdog'].isin(mandatory_list))]
mandatory_picks

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score


In [7]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Home Team,Away Team,Favorite,Underdog,Spread,Total,Implied Score


In [8]:
update_google_sheet(short_df)

05:18:15 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
05:18:15 INFO:Obtained credentails from existing 'token.json' file
05:18:15 INFO:Writing data for Hoke
05:18:15 INFO:file_cache is only supported with oauth2client<4.0.0
05:18:15 DEBUG:Attempting to write 5 rows to Hoke!A:K
05:18:15 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1bxqOme4RHBUMgc5ojKr-RgGt3vfetGDAZ1sfs0SeWw4/values/Hoke%21A%3AK:append?valueInputOption=USER_ENTERED&alt=json
05:18:15 INFO:Appended 50 cells to Hoke!A:K
05:18:15 INFO:Writing data for Lou
05:18:15 INFO:file_cache is only supported with oauth2client<4.0.0
05:18:15 DEBUG:Attempting to write 5 rows to Lou!A:K
05:18:15 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1bxqOme4RHBUMgc5ojKr-RgGt3vfetGDAZ1sfs0SeWw4/values/Lou%21A%3AK:append?valueInputOption=USER_ENTERED&alt=json
05:18:15 INFO:Appended 50 cells to Lou!A:K
05:18:15 INFO:Writing data for Tom
05:18:15 INFO:file_cach